In [ ]:
import csv

dataset_path = './data/fer/'

train_csv = dataset_path + 'train.csv'
test_csv = dataset_path + 'test.csv'
val_csv = dataset_path + 'val.csv'
csv_file = dataset_path + 'fer2013.csv'

In [ ]:
# Building model

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, Activation, BatchNormalization
import tensorflow as tf


# Build the model
def build_model(self):
    gpus = tf.config.list_physical_devices('GPU')
    print(gpus[0])
    with tf.device("/gpu:0"):
        self.model = Sequential()
        self.model.add(Conv2D(32, (3,3), activation='relu', input_shape=(48, 48, 1)))
        self.model.add(MaxPooling2D((2, 2)))
        self.model.add(Conv2D(64, (3,3), activation='relu'))
        self.model.add(MaxPooling2D((2, 2)))
        self.model.add(Dropout(0.5))
        self.model.add(Flatten())
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dense(7, activation='softmax'))


In [ ]:
# Train the model

from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from tqdm import tqdm

def train_model(self, extra_callbacks=[]):

    root_path = self.root_path
    learning_rate = self.learning_rate
    batch_size= self.batch_size
    epochs = self.epochs
    img_size = self.img_size
    postfix = self.postfix

    sgd = SGD(lr=learning_rate, decay=1e-6, momentum=0.9, nesterov=True)
    adam = Adam(learning_rate)
    self.model.compile(
        loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"]
    )

    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True
    )

    val_datagen = ImageDataGenerator(rescale=1.0 / 255)
    eval_datagen = ImageDataGenerator(rescale=1.0 / 255)
    train_generator = train_datagen.flow_from_directory(
        root_path + "/train"+postfix,
        target_size=(img_size, img_size),
        color_mode="grayscale",
        batch_size=batch_size,
        class_mode="categorical"
    )

    val_generator = val_datagen.flow_from_directory(
        root_path + "/val"+postfix,
        target_size=(img_size, img_size),
        color_mode="grayscale",
        batch_size=batch_size,
        class_mode="categorical"
    )

    eval_generator = eval_datagen.flow_from_directory(
        root_path + "/test"+postfix,
        target_size=(img_size, img_size),
        color_mode="grayscale",
        batch_size=batch_size,
        class_mode="categorical"
    )

    print(val_generator.class_indices)

    es_cb = EarlyStopping(monitor='val_accuracy',
        min_delta=0.00005,
        patience=11,
        verbose=1,
        restore_best_weights=True,
    )
    cp_cb = ModelCheckpoint(
        filepath='model.h5',
        save_best_only=True,
    )

    history_fit = self.model.fit(
        train_generator,
        steps_per_epoch=800/(batch_size/32), #28709
        epochs=epochs,
        # validation_steps=2000,
        validation_data=val_generator,
        callbacks=[es_cb] + extra_callbacks,
    )

    history_predict = tqdm(self.model.evaluate(eval_generator, steps=2000))

    #draw the graph
    plot_history(history_fit)

def plot_history(history):
    plt.plot(history.history['accuracy'], label='train_accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.legend()
    plt.title('Accuracy Plot')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.show()

    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.legend()
    plt.title('Loss Plot')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()


In [ ]:
# Save the model

def save_model(self):
    root_path = self.root_path + '/fine-tune'
    model_json = self.model.to_json()
    with open(root_path+"/model.json", "w") as json_file:
        json_file.write(model_json)
    self.model.save_weights(root_path+"/model_weight.h5")
    self.model.save(root_path+"/model.h5")

In [ ]:
import tensorflow as tf;  
gpus = tf.config.list_physical_devices('GPU')
print(gpus)
if gpus: 
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=5292)]
    )

In [ ]:

import wandb

# Const variables
img_size = 48
num_classes = 7
root_path = "./data/fer"
batch_size = 64
epochs = 500
learning_rate=0.001


In [ ]:
import wandb
wandb.login()


In [ ]:


class Model:
    img_size = 48
    num_classes = 7
    root_path = "./data/fer"
    batch_size = 64
    epochs = 100
    learning_rate=0.001
    postfix = ""

    def __init__(self):
        self.model = None

    def build_model(self):
        build_model(self)

    def train_model(self, another_callback=[]):
        train_model(self, another_callback)

    def save_model(self):
        save_model(self)



In [ ]:
import scipy
import tensorflow as tf
from wandb.keras  import WandbCallback

if __name__ == "__main__":
    img_size = 48
    num_classes = 7
    root_path = "./data/fer"
    batch_size = 64
    epochs = 100
    learning_rate=0.001
    postfix = ""
    wandb.init(project="fyp-fer-fine-tune", entity="johnkhw")
    wandb.config = {
        learning_rate,
        epochs,
        batch_size
    }
    wandb_callback = [
        WandbCallback(),
    ]
    model = Model()
    model.build_model()
    model.train_model(wandb_callback)
    model.save_model()
    wandb.finish()

In [2]:

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, Activation, BatchNormalization
import tensorflow as tf
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import os

from tqdm import tqdm
import wandb
wandb.login()


C:\Users\wongk\AppData\Local\Temp\ipykernel_12892\326642752.py:9: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


True

In [3]:
def build_model(hp):
    model = Sequential()
    # Conv2D layer
    model.add(Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                     kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
                     activation='relu',
                     input_shape=(48, 48, 1)))
    model.add(MaxPooling2D((2, 2)))
    # Conv2D layer
    model.add(Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(hp.Float('dropout_1_rate', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Flatten())
    # Dense layer
    model.add(Dense(units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16), activation='relu'))
    model.add(Dense(7, activation='softmax'))
    # Optimizer
    optimizer_choice = hp.Choice('optimizer', ['sgd', 'adam'])
    if optimizer_choice == 'sgd':
        optimizer = SGD(lr=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG'), decay=1e-6, momentum=0.9,nesterov=True)
    else:
        optimizer = Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG'))
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model


In [4]:
import os
import csv
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import wandb
from wandb.keras import WandbCallback
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

# Set GPU memory limit
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=5292)]
    )

# Dataset path
dataset_path = './data/fer/'
train_csv = dataset_path + 'train.csv'
test_csv = dataset_path + 'test.csv'
val_csv = dataset_path + 'val.csv'
csv_file = dataset_path + 'fer2013.csv'

# Load data generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)
eval_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    dataset_path + "/train",
    target_size=(48, 48),
    color_mode="grayscale",
    batch_size=64,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    dataset_path + "/val",
    target_size=(48, 48),
    color_mode="grayscale",
    batch_size=64,
    class_mode="categorical"
)


Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
os.environ['WANDB_SILENT'] = 'true'


In [6]:
if __name__ == "__main__":
    img_size = 48
    num_classes = 7
    root_path = "./data/fer"
    batch_size = 64
    wandb.init(project="fyp-fer-fine-tune", entity="johnkhw")
    # Hyperparameter tuning
    tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=10,executions_per_trial=3,directory='output',project_name='fyp-fer-fine-tune',overwrite=True)
    tuner.search(train_generator,
                 steps_per_epoch=800/(batch_size/32),
                 epochs=100,
                 validation_data=val_generator,
                 callbacks=[WandbCallback(), EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=11, verbose=0, restore_best_weights=True)])
    # Get the best model and its hyperparameters
    best_model = tuner.get_best_models(num_models=1)[0]
    best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
    print("Best hyperparameters:")
    print(best_hyperparameters)
    
    # Save the best model with a versioned file name
    model_version = 1
    model_path = os.path.join(root_path, f'model_v{model_version}.h5')
    while os.path.exists(model_path):
        model_version += 1
        model_path = os.path.join(root_path, f'model_v{model_version}.h5')

    best_model.save(model_path)
    print(f"Best model saved as: {model_path}")

    # Evaluate the best model on the test set
    eval_generator = eval_datagen.flow_from_directory(
        root_path + "/test",
        target_size=(img_size, img_size),
        color_mode="grayscale",
        batch_size=batch_size,
        class_mode="categorical"
    )
    results = best_model.evaluate(eval_generator, steps=2000)
    print(f"Test set evaluation results: {results}")

    wandb.finish()

Trial 10 Complete [01h 15m 03s]
val_accuracy: 0.4524008532365163

Best val_accuracy So Far: 0.5830779075622559
Total elapsed time: 07h 51m 12s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Best hyperparameters:
Best model saved as: ./data/fer\model_v1.h5
Found 3589 images belonging to 7 classes.
  55/2000 [..............................] - ETA: 1:11 - loss: 1.1728 - accuracy: 0.5872WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 2000 batches). You may need to use the repeat() function when building your dataset.


2000/2000 [==============================] - 3s 1ms/step - loss: 1.1730 - accuracy: 0.5865
Test set evaluation results: [1.1729532480239868, 0.5865143537521362]


In [8]:
# show moel parameters
best_model.summary()

# load model
old_model = keras.models.load_model('./data/fer/model.h5')
old_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 44, 44, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 18, 18, 32)        51232     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 9, 9, 32)         0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 9, 9, 32)          0         
                                                                 
 flatten (Flatten)           (None, 2592)              0

In [10]:
# show tuning results and all parameters
tuner.results_summary()

# show best hyperparameters
best_hyperparameters.values



Results summary
Results in output\fyp-fer-fine-tune
Showing 10 best trials
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 5
conv_2_filter: 32
conv_2_kernel: 5
dropout_1_rate: 0.1
dense_1_units: 128
optimizer: adam
learning_rate: 0.00036156986755960413
Score: 0.5830779075622559
Trial summary
Hyperparameters:
conv_1_filter: 128
conv_1_kernel: 3
conv_2_filter: 48
conv_2_kernel: 3
dropout_1_rate: 0.1
dense_1_units: 48
optimizer: adam
learning_rate: 0.00026152978239516096
Score: 0.569239338239034
Trial summary
Hyperparameters:
conv_1_filter: 112
conv_1_kernel: 5
conv_2_filter: 32
conv_2_kernel: 3
dropout_1_rate: 0.2
dense_1_units: 96
optimizer: adam
learning_rate: 0.00010958136414612152
Score: 0.5652456680933634
Trial summary
Hyperparameters:
conv_1_filter: 128
conv_1_kernel: 5
conv_2_filter: 64
conv_2_kernel: 3
dropout_1_rate: 0.1
dense_1_units: 48
optimizer: adam
learning_rate: 0.0021043482724264983
Score: 0.530881385008494
Trial summary
Hyperparameters:
conv_1_filter: 32

{'conv_1_filter': 64,
 'conv_1_kernel': 5,
 'conv_2_filter': 32,
 'conv_2_kernel': 5,
 'dropout_1_rate': 0.1,
 'dense_1_units': 128,
 'optimizer': 'adam',
 'learning_rate': 0.00036156986755960413}